In [1]:
# run w aln output in both modes
import sys
sys.path.append('../src/')

#matcAFDB_tools up positions


#calc fident with cross alphabet comparison




In [19]:
import subprocess ,shlex


def runargs(args):
    '''run a command line command

    Parameters
    ----------
    args : str
        command line command
    '''

    args = shlex.split( args)
    p = subprocess.run( args )
    return p

def runFoldseek_allvall( structfolder , outfolder , foldseekpath = '../foldseek/foldseek' , mode = 2):
    '''
    run foldseek search and createtsv

    parameters
    ----------
    dbpath : str
        path to foldseek database
    outfolder : str 
        path to output folder
    maxseqs : int   
        maximum number of sequences to compare to

    '''

    args = foldseekpath + " easy-search " + structfolder + " "+ structfolder +" "+ outfolder+"aln_"+str(mode)+"_allvall.csv tmp" + " --format-output 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,alntmscore' --exhaustive-search --alignment-type "+ str(mode) 
    p = runargs(args)
    return outfolder+"aln_"+str(mode)+"_allvall.csv"

def read_dbfiles3di(folder, name = 'outdb'):
    #find positions 
    threeDidb = folder+ name +'_ss'
    threeDiseq = [ l.strip().replace('\x00','') for l in open(threeDidb)]
    
    
    lookup = folder+name+'.lookup'
    ids = [ l.split()[1].strip() for l in open(lookup)]
    
    AADB = folder+name
    AAs = [ l.strip().replace('\x00','') for l in open(AADB)]

    mapper3di = dict(zip(ids,threeDiseq))
    mapperAA = dict(zip(ids,AAs))
    
    return mapper3di, mapperAA

In [21]:
mapper3di, mapperAA = read_dbfiles3di('../testdata/')


In [22]:
structfolder = '../testdata/structs'
runFoldseek_allvall(structfolder , outfolder = structfolder, mode = 0)
runFoldseek_allvall(structfolder , outfolder = structfolder, mode = 2)

../testdata/structsaln_0_allvall.csv exists and will be overwritten
easy-search ../testdata/structs ../testdata/structs ../testdata/structsaln_0_allvall.csv tmp --format-output query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,alntmscore --exhaustive-search --alignment-type 0 

MMseqs Version:              	30fdcac78217579fa25d59bc271bd4f3767d3ebb
Seq. id. threshold           	0
Coverage threshold           	0
Coverage mode                	0
Max reject                   	2147483647
Max accept                   	2147483647
Add backtrace                	false
TMscore threshold            	0
TMalign hit order            	0
TMalign fast                 	1
Preload mode                 	0
Threads                      	48
Verbosity                    	3
LDDT threshold               	0
Sort by structure bit score  	1
Substitution matrix          	aa:3di.out,nucl:3di.out
Alignment mode               	3
Alignment mode               	0
E-value th

'../testdata/structsaln_2_allvall.csv'

In [23]:
import pandas as pd

In [25]:
alndf0 = pd.read_table('../testdata/structsaln_0_allvall.csv' , header = None)
alndf2 = pd.read_table('../testdata/structsaln_2_allvall.csv', header = None)
columns = 'query,target,fident,alnlen,mismatch,gapopen,qstart,qend,tstart,tend,evalue,bits,lddt,qaln,taln,cigar,lntmscore'.split(',')
alndf0.columns = columns
alndf2.columns = columns

alndf0['3diq']= alndf0['query'].map(mapper3di)
alndf0['3dit']= alndf0['target'].map(mapper3di)
alndf0['AAq']= alndf0['query'].map(mapperAA)
alndf0['AAt']= alndf0['target'].map(mapperAA)


alndf0 = alndf0.set_index(['query', 'target'])
alndf2 = alndf2.set_index(['query', 'target'])

alndf = alndf0.merge(alndf2, left_index=True, right_on=['query', 'target'])


In [34]:
#use the qaln and taln of x and y to derive the other side fident
import numpy as np
def Fident(str1,str2 , verbose = False):
    #minlen= min( (len(str1),len(str2))  )
    #str1 = str1[:minlen]
    #str2 = str2[:minlen]
    
    str1 = np.array(list(str1))
    str2 = np.array(list(str2))            
 
    return len(np.where( (str1 == str2 ) & (str1 != '-' ) & (str2 != '-')  )[0]) / len(str1)


def copyaln( aln, seq):
    seqiter = iter(seq)
    newaln = ''
    
    minlen= min( (len(aln.replace('-','')),len(seq))  )
    for i,char in enumerate(aln):
        if i > minlen-1:
            break
        if char == '-':
            newaln += '-'
        else:
            newaln+=next(seqiter)
        
    return newaln

def calc_fident_crossaln(row , verbose = False):
    #amino acid representations of alns using AAand3di or just 3di
    qaln_0,taln_0, qaln_2, taln_2 = row.qaln_x, row.qaln_y, row.taln_x, row.taln_y
    #start and stop of aln
    qstart_0,qend_0, tstart_0 , tend_0, qstart_2, qend_2, tstart_2 , tend_2 = row.qstart_x, row.qend_x , row.tstart_x , row.tend_x , row.qstart_y , row.qend_y , row.tstart_y , row.tend_y
    #indexing starts at 1...
    
    #3di of the query and target
    structQ, structT = row['3diq'], row['3dit']
    AAq, AAt = row['AAq'], row['AAt'] 

    #add gaps
    qAA_newgaps = copyaln(qaln_0, structQ[qstart_0-1:qend_0-1])
    q3di_newgaps = copyaln(qaln_2, structQ[qstart_2-1:qend_2-1])
    tAA_newgaps = copyaln(taln_0, structT[tstart_0-1:tend_0-1])
    t3di_newgaps = copyaln(taln_2, structT[tstart_2-1:tend_2-1]) 
    
    if verbose == True:
        print('q3di->AA', qAA_newgaps )
        print('t3di->AA', tAA_newgaps )
        print('qAA3di ->3di', q3di_newgaps )
        print('tAA3di ->3di', t3di_newgaps )

    #return columns
    return [ Fident(q3di_newgaps , t3di_newgaps , verbose = verbose) , Fident(qAA_newgaps , tAA_newgaps, verbose = verbose)]

In [35]:
testrow = alndf.iloc[121]
print(testrow)
calc_fident_crossaln(testrow, verbose = True)

fident_x                                                   0.836
alnlen_x                                                     773
mismatch_x                                                   121
gapopen_x                                                      3
qstart_x                                                       1
qend_x                                                       770
tstart_x                                                       1
tend_x                                                       770
evalue_x                                                     0.0
bits_x                                                       813
lddt_x                                                    0.8428
qaln_x         MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...
taln_x         MLPGLALLLLAAWTARALEVPTDGNAGLLAEPQIAMFCGRLNMHMN...
cigar_x                                      603M2D84M3I52M1D28M
lntmscore_x                                                0.342
3diq           DDDDDDDDDD

[0.8010403120936281, 0.8153446033810143]

In [36]:
alndf['Fidentx'] = alndf.apply(calc_fident_crossaln , axis =1 )
print(alndf)

ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (768,) and arg 1 with shape (769,).